In [1]:
import  numpy as np
import json

In [11]:
np.random.exponential(100000000)

213913921.82812425

In [21]:
with open('test.json', 'r') as f:
    data = json.load(f)


In [22]:
import scipy.io
scipy.io.savemat('test.mat', data)

In [23]:
a  =  np.random.rand(3) / 3

In [24]:
import numpy as np
import pickle
import time
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal as mvn
from scipy.stats import skewnorm
from scipy.stats import norm
from scipy.sparse import csr_matrix
from hkstools.hksimulation.Simulation_Branch_HP import Simulation_Branch_HP
from hkstools.Initialization_Cluster_Basis import Initialization_Cluster_Basis
from hkstools.Learning_Cluster_Basis import Learning_Cluster_Basis
from hkstools.Estimate_Weight import Estimate_Weight
from hkstools.Loglike_Basis import Loglike_Basis
from hkstools.DistanceSum_MPP import DistanceSum_MPP
from hkstools.Kernel_Integration import Kernel_Integration
from hkstools.hksimulation.Kernel import Kernel
from scipy.special import erf

In [25]:
np.random.seed(0)

###################simulation hks data#######################
options = {
    'N': 100, 'Nmax': 100, 'Tmax': 50, 'tstep': 0.1,
    'dt': [0.1], 'M': 250, 'GenerationNum': 10
}
D = 3   # event type个数
K = 2  # cluster个数
nTest = 5
nSeg = 5
nNum = options['N'] / nSeg
mucenter = np.random.rand(D) / D
mudelta = 0.05
# First01 cluster: Hawkes process with exponential kernel

print('01 Simple exponential kernel')
para1 = {'kernel': 'exp', 'landmark': [0]}
para1['mu'] = mucenter+2
L = len(para1['landmark'])
para1['A'] = np.zeros((D, D, L))
for l in range(1, L + 1):
    para1['A'][:, :, l - 1] = (0.7**l) * np.random.rand(D,D)
eigvals_list = []
eigvecs_list = []
for l in range(L):
    eigvals, eigvecs = np.linalg.eigh(para1['A'][:, :, l])
    eigvals_list.append(eigvals)
    eigvecs_list.append(eigvecs)
all_eigvals = np.concatenate(eigvals_list)
max_eigval = np.max(all_eigvals)
para1['A'] = 0.5 * para1['A'] / max_eigval
para1['w'] = 0.5
Seqs1 = Simulation_Branch_HP(para1, options)
# First02 cluster: Hawkes process with exponential kernel
print('02 Simple exponential kernel')
para2 = {'kernel': 'exp', 'landmark': [0]}
para2['mu'] = mucenter
L = len(para2['landmark'])
para2['A'] = np.zeros((D, D, L))
for l in range(1, L + 1):
    para2['A'][:, :, l - 1] = (0.7**l) * np.random.rand(D,D)
eigvals_list = []
eigvecs_list = []
for l in range(L):
    eigvals, eigvecs = np.linalg.eigh(para2['A'][:, :, l])
    eigvals_list.append(eigvals)
    eigvecs_list.append(eigvecs)
all_eigvals = np.concatenate(eigvals_list)
max_eigval = np.max(all_eigvals)
para2['A'] = 0.5 * para2['A'] / max_eigval
para2['w'] = 0.5
Seqs2 = Simulation_Branch_HP(para2, options)


'''
# Second cluster: Hawkes process with Gaussian kernel
print('Complicated gaussian kernel')
para2 = {'kernel': 'gauss', 'landmark': np.arange(0, 13, 3)}
para2['mu'] = np.random.rand(D) / D
L = len(para2['landmark'])
para2['A'] = np.zeros((D, D, L))
for l in range(1, L + 1):
    para2['A'][:, :, l - 1] = (0.9**l) * np.random.rand(D,D)
para2['A'] = 0.25 * para2['A'] / np.max(np.abs(np.linalg.eigh(np.sum(para2['A'], axis=2))[0]))
para2['A'] = np.reshape(para2['A'], (D, L, D))
#import pdb;pdb.set_trace()
para2['w'] = 1
Seqs2 = Simulation_Branch_HP(para2, options)

'''
SeqsMix = Seqs1 + Seqs2

01 Simple exponential kernel
#seq=10/100, #event=320
#seq=20/100, #event=372
#seq=30/100, #event=317
#seq=40/100, #event=337


KeyboardInterrupt: 

In [ ]:
Seqs1[5]['Time'].shape

(356,)

In [ ]:
Seqs2[5]['Time'].shape

(39,)

In [ ]:
a+2

array([2.01108838, 2.03467399, 2.28296196])

In [2]:
with open('SeqsMix.json', 'r') as file:
    SeqsMix = json.load(file)   
for c in range(len(SeqsMix)):
    SeqsMix[c]['Time'] = np.array(SeqsMix[c]['Time'])
    SeqsMix[c]['Mark'] = np.array(SeqsMix[c]['Mark'])
    SeqsMix[c]['Mark'] = SeqsMix[c]['Mark'] - 1

In [11]:
with open('test.json', 'r') as file:
    SeqsMix = json.load(file)   
SeqsMix['Time'] = np.array(SeqsMix['Time'])
SeqsMix['Mark'] = np.array(SeqsMix['Mark'])
SeqsMix=[SeqsMix]

In [13]:
len(SeqsMix[0]['Time'])

366

In [3]:
from HawkesModel import HawkesModel
hakes_model = HawkesModel(SeqsMix,2,Tmax=50)

In [4]:
len(hakes_model.Seqs)

200

In [5]:
N = len(hakes_model.Seqs)
D = np.zeros(N)
for i in range(N):
    D[i] = np.max(hakes_model.Seqs[i]['Mark'])
D = int(np.max(D)) + 1


sigma = np.zeros(N)
Tmax = np.zeros(N)

for i in range(N):
    sigma[i] = ((4 * np.std(hakes_model.Seqs[i]['Time'])**5) / (3 * len(hakes_model.Seqs[i]['Time'])))**0.2
    Tmax[i] = hakes_model.Seqs[i]['Time'][-1] + np.finfo(float).eps
Tmax = np.mean(Tmax)

hakes_model.kernel = 'gauss'  # 核函数类型
#hakes_model.w = np.mean(sigma)  # 带宽
hakes_model.w = 7.064599620296471
#hakes_model.landmark = hakes_model.w * np.arange(0, np.ceil(Tmax / hakes_model.w))  # 地标
hakes_model.landmark = [ 0.        ,  7.06459962, 14.12919924, 21.19379886, 28.25839848,35.3229981 , 42.38759772]
hakes_model.Tmax= 50

hakes_model.K = hakes_model.clusternum
hakes_model.D = D
hakes_model.alpha = 1
hakes_model.M = len(hakes_model.landmark)
hakes_model.beta_a = np.ones((D, hakes_model.M, hakes_model.K, D)) / (hakes_model.M * D**2)   # kernel前的系数 accidk D M K D
hakes_model.beta_na = np.ones((D, hakes_model.M, 1, D)) / (hakes_model.M * D**2)
hakes_model.b_a = np.ones((D, hakes_model.K)) / D     # base intensity shape = [event_type , k]
hakes_model.b_na = np.ones((D, 1))/D
label = np.concatenate([np.zeros(100),np.ones(100)])
hakes_model.label = label

In [6]:
hakes_model.D

3

In [41]:
hakes_model.w

7.064599620296471

In [23]:
hakes_model.kernel

'gauss'

In [40]:
hakes_model.landmark

[0.0,
 7.06459962,
 14.12919924,
 21.19379886,
 28.25839848,
 35.3229981,
 42.38759772]

In [43]:
hakes_model.beta_a[:,:,0,:]

array([[[0.01587302, 0.01587302, 0.01587302],
        [0.01587302, 0.01587302, 0.01587302],
        [0.01587302, 0.01587302, 0.01587302],
        [0.01587302, 0.01587302, 0.01587302],
        [0.01587302, 0.01587302, 0.01587302],
        [0.01587302, 0.01587302, 0.01587302],
        [0.01587302, 0.01587302, 0.01587302]],

       [[0.01587302, 0.01587302, 0.01587302],
        [0.01587302, 0.01587302, 0.01587302],
        [0.01587302, 0.01587302, 0.01587302],
        [0.01587302, 0.01587302, 0.01587302],
        [0.01587302, 0.01587302, 0.01587302],
        [0.01587302, 0.01587302, 0.01587302],
        [0.01587302, 0.01587302, 0.01587302]],

       [[0.01587302, 0.01587302, 0.01587302],
        [0.01587302, 0.01587302, 0.01587302],
        [0.01587302, 0.01587302, 0.01587302],
        [0.01587302, 0.01587302, 0.01587302],
        [0.01587302, 0.01587302, 0.01587302],
        [0.01587302, 0.01587302, 0.01587302],
        [0.01587302, 0.01587302, 0.01587302]]])

In [7]:
hakes_model.b_a

array([[0.33333333, 0.33333333],
       [0.33333333, 0.33333333],
       [0.33333333, 0.33333333]])

In [8]:
np.where(hakes_model.label==0)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
        51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
        68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
        85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]),)

In [9]:
np.where(label==0)[1]

IndexError: tuple index out of range

In [10]:
hakes_model.Loglike(0)

IndexError: tuple index out of range

In [12]:
from scipy.io import savemat
data_dict = {'mylist': SeqsMix}
savemat('list.mat', data_dict)

In [13]:
fields = {key for d in SeqsMix for key in d.keys()}

# 然后，确保每个字典都有所有的键
structured_data = [{field: d.get(field, None) for field in fields} for d in SeqsMix]

# 保存为.mat文件
savemat('data.mat', {'my_struct_array': structured_data})

In [ ]:
hakes_model.landmark

[0.0,
 7.06459962,
 14.12919924,
 21.19379886,
 28.25839848,
 35.3229981,
 42.38759772]

In [10]:
hakes_model.loglike_one_a(SeqsMix[0],0)

-125.40594078305216

In [ ]:
hakes_model.w

7.064599620296471

In [ ]:
np.where(np.array)

In [12]:
np.ones((3,8,3))

array([[[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]]])

In [14]:
a=[0.1061166 ,  0.22921302,  0.30268615,  0.37942205,  0.82141034,
        1.03059649,  1.14252087,  1.15234558,  1.15506899,  1.35584022,
        1.86853647,  1.9510604 ,  1.96784365,  1.98847428,  2.08685125,
        2.12782196,  2.20903295,  2.21156209,  2.33782832,  2.72155033,
        2.78095279,  2.94043383,  3.0869555 ,  3.11844972,  3.16897799,
        3.28156836,  3.87550134,  3.90674154,  4.04790724,  4.13163278,
        4.15471644,  4.29712607,  4.32631015,  4.5556877 ,  4.79833023,
        5.29820109,  5.79979138,  5.97902408,  6.02334544,  6.07016892,
        6.12119572,  6.13003537,  6.24151504,  6.3402829 ,  6.45466899,
        6.50581035,  6.52465445,  6.57020833,  6.68795887,  6.92339102,
        7.0744089 ,  7.25157662,  7.28971275,  7.4092582 ,  7.4429142 ,
        7.52193074,  7.68222921,  7.90635217,  8.19028542,  8.47492948,
        8.64718405,  9.06391315,  9.13743262,  9.14001659,  9.28392595,
        9.41224262,  9.43165249,  9.54417095,  9.65804824,  9.79882819,
       10.10131272, 10.17764093, 10.39642727, 10.41050896, 10.57752329,
       10.59910597, 10.62274938, 10.64037388, 10.85993642, 10.92965243,
       11.08904424, 11.25975546, 11.75441683, 11.75598793, 11.93056105,
       12.02871186, 12.05846376, 12.26891879, 12.3254064 , 12.48791264,
       12.51190695, 12.62669677, 12.9895432 , 13.09182049, 13.26645854,
       13.33417389, 13.36164483, 13.54102448, 13.57542619, 13.58339698,
       13.63321962, 13.69647513, 13.69980691, 13.85147871, 13.95402801,
       14.57264837, 14.7177023 , 14.72048459, 14.77190962, 14.89024321,
       15.02244378, 15.06505373, 15.09243343, 15.24751521, 15.64011257,
       15.67917276, 15.77653645, 15.80753979, 15.87002998, 15.91339212,
       16.17926402, 16.27640053, 16.44464222, 16.55575932, 16.57662275,
       16.63521681, 16.82849078, 17.14055637, 17.24767527, 17.68347478,
       17.72021183, 17.72242065, 17.87020862, 17.91432993, 17.9231422 ,
       18.42901207, 18.48404271, 18.51918633, 18.89740502, 19.03013351,
       19.07636728, 19.20463554, 19.26150698, 19.79590029, 19.88762609,
       19.9489316 , 20.17914938, 20.27462209, 20.2883512 , 20.76954333,
       21.08559501, 21.13961939, 21.20934841, 21.22829848, 21.4007182 ,
       21.59701208, 21.60806039, 21.75671204, 21.82946641, 22.09214075,
       22.13416209, 22.14175763, 22.18228717, 22.33558602, 22.3800398 ,
       22.4066872 , 22.41449092, 22.61558629, 22.65562963, 22.77373105,
       22.83551639, 22.91763896, 23.13209467, 23.22963405, 23.34452305,
       23.4828224 , 23.55786576, 23.66029954, 23.70365565, 23.77014796,
       23.79777265, 23.90241759, 24.18740714, 24.35913541, 24.67165136,
       24.83201319, 25.02051763, 25.05711277, 25.26932025, 25.35101965,
       25.61101657, 26.01554225, 26.24475149, 26.47011615, 26.60228675,
       26.61092218, 26.65083263, 26.65534888, 26.71389247, 26.71608758,
       26.78447784, 26.79846888, 27.06997477, 27.12271685, 27.24979985,
       27.25189707, 27.26129733, 27.2946241 , 27.3133542 , 27.3297162 ,
       27.81765779, 27.88779523, 28.02355758, 28.62993412, 28.83379264,
       28.87063333, 28.90829289, 29.0054099 , 29.11327386, 29.27675816,
       29.33637681, 29.68168696, 29.71699126, 29.9419537 , 30.40500113,
       30.45190345, 30.49015799, 30.89215377, 31.04821187, 31.22314027,
       31.26554755, 31.32806463, 31.36427207, 31.44381206, 31.68748223,
       31.78054057, 31.91455599, 32.29014422, 32.45048519, 33.14183582,
       33.15163667, 33.17367574, 33.19244176, 33.25678512, 33.71486824,
       33.75771361, 34.06047496, 34.16755987, 34.24859784, 34.28667369,
       34.33622246, 34.43547289, 34.48956974, 34.75394475, 34.7929515 ,
       34.80768695, 34.98734679, 35.11719727, 35.14777289, 35.29814095,
       35.40274851, 35.40684208, 35.40789746, 35.50871963, 35.52122035,
       35.52450217, 35.6542985 , 35.68568595, 35.80044791, 35.94906271,
       35.94942363, 36.07172654, 36.51047974, 36.66945517, 36.76451604,
       36.97313691, 37.0509316 , 37.20880197, 37.2432632 , 37.37343547,
       37.50203544, 38.02756326, 38.1207104 , 38.2246484 , 38.46082885,
       38.79723117, 38.82284392, 39.22811479, 39.25143616, 39.33027653,
       39.35356024, 39.43093218, 39.58989915, 39.59528869, 39.60369182,
       39.78324212, 39.93211127, 39.98051755, 40.04047933, 40.10384059,
       40.12943498, 40.28150491, 40.40611563, 40.45065897, 40.54700594,
       40.62830756, 41.09806106, 41.11163648, 41.2308506 , 41.29835246,
       41.35617906, 41.49150692, 41.99006006, 42.18290107, 42.23739676,
       42.24602124, 42.32150518, 42.51926163, 42.53590071, 42.65468122,
       42.9059965 , 43.45265446, 43.51646466, 43.67046127, 43.93458864,
       44.02641675, 44.06328178, 44.32503155, 44.40983828, 44.49127217,
       44.56475723, 44.58139557, 44.65035116, 44.798733  , 44.96552615,
       45.03344995, 45.16055475, 45.39098933, 45.56377304, 45.57938238,
       45.64869197, 45.65274637, 45.66825193, 45.83021936, 46.25657386,
       46.52572903, 46.62780091, 46.72683075, 46.75502024, 46.85236984,
       46.91465694, 46.92585161, 46.95921871, 47.00027722, 47.00932467,
       47.03290876, 47.23088397, 47.25308266, 47.32873175, 47.38605471,
       47.57143995, 47.57535728, 47.64182612, 47.79832182, 47.98820596,
       48.08090939, 48.14144159, 48.23279093, 48.27609984, 48.29274795,
       48.29913958, 48.29965529, 48.4261868 , 48.71101266, 49.16340343,
       49.21103701, 49.56717366, 49.60862594, 49.67063737, 49.70858406,
       49.96957318]

In [16]:
len(a)

381